[爬取对象：https://www.yuque.com/kafangxunlianying/qndurp/](https://www.yuque.com/kafangxunlianying/qndurp/)

# 几个库

In [1]:
import requests
import json
import re
import time
import os

import parsel
import pandas as pd
from urllib.parse import unquote

注意到每个试卷的页面的源代码都会有一大段的加密数据：
![](./assets/WechatIMG57.png)

图中的这个函数`decodeURIComponent`可以在python中找到如下的替代：

（参考了[stackoverflow上的博客](https://stackoverflow.com/questions/47749168/mimic-js-decodeuricomponent-and-unescape-in-python)

In [2]:
# decode twice because the string has been encoded twice.
unquote(unquote('%25D7%2590%25D7%2591%25D7%2592'))

'אבג'

# 获取列表

In [3]:
def get_html(url):
    """获取网页文件"""
    response = requests.get(url)
    response.encoding = "uft8"
    html = response.text
    return html

In [4]:
def get_response(html):
    """把返回的网页中的加密数据解码，实际上返回值是一个json文件"""
    selector = parsel.Selector(html)
    l = selector.css("script").getall()[1]
    pattern = "decodeURIComponent\(\"(.*)\"\)"
    s = re.findall(pattern,l)[0]
    s = unquote(s)
    d = json.loads(s)
    return d

In [5]:
html = get_html("https://www.yuque.com/kafangxunlianying/qndurp/hv1nhp")
d = get_response(html)

In [18]:
d.keys()

dict_keys(['me', 'notification', 'settings', 'env', 'space', 'isYuque', 'isPublicCloud', 'isEnterprise', 'defaultSpaceHost', 'timestamp', 'traceId', 'siteName', 'siteTip', 'topTip', 'readTip', 'imageServiceDomains', 'sharePlatforms', 'locale', 'matchCondition', 'customTracertConfig', 'group', 'book', 'doc', 'isDocCollaborator', 'isCollaborationPage', 'backgroundImage', 'forbidLoginCard', 'loginCardPV', 'docUrl', 'search', 'prefetch', 'enableMobileAppRelease', 'isRunAtRaw', 'enableUserFeed', 'paymentInfo', 'login', 'isDesktopApp', 'isAlipayApp', 'isDingTalkApp', 'isDingTalkMiniApp', 'isDingTalkDesktopApp', 'isYuqueMobileApp', 'tracertConfig'])

In [6]:
# 这是一个常量，对于我们的目标网页而言
d['book']['id']

20700413

In [7]:
# 能用到的一些变量
### level：页面的层级
### type：页面的种类
### url：页面的地址（https://www.yuque.com/kafangxunlianying/qndurp/hv1nhp/{url}）
### title：页面的标题
df = pd.DataFrame(d['book']['toc'])[['level','type','url','title']]
df

,level,type,url,title
0,0,TITLE,,茆pro派
1,1,TITLE,,真题解析
2,2,DOC,ukh065,茆pro派《五年考研三年模拟》习题集请关注“卡方训练营”公众号咨询小助手
3,1,TITLE,,北京大学 431
4,2,DOC,tmi64a,2022年北京大学431金融数学真题
...,...,...,...,...
691,2,DOC,dgt9ik,2019年西安交通大学432应用统计真题
692,2,DOC,lesxpw,2018年西安交通大学432应用统计真题
693,2,DOC,ddd0t4,2017年西安交通大学432应用统计真题
694,2,DOC,slxmv6,2016年西安交通大学432应用统计真题


# 批量抓取

In [8]:
def replace_cards(s):
    """把html文件中的<card></card>标签解密、转换为markdown的$$标签"""
    
    def md(card):
        """解码markdown公式"""
        s = unquote(card)[5:]
        code = json.loads(s)['code']
        return f"${code}$"

    selector = parsel.Selector(s)
    cards = selector.css('card').getall()
    values = selector.css('card::attr(value)').getall()
    for card, value in zip(cards, values):
        s = s.replace(card, md(value))
    return s

In [9]:
# 一些html转换为markdown的对应关系
labels = ['li', 'ol', 'p', 'span', 'meta', 'h1', 'h2', 'h3', 'h4']
prefixes = ['- ', '', '', '', '', '# ', '## ', '### ', '#### ']
suffixes = ['\n', '', '\n ', '', '', '\n', '\n', '\n', '\n']
# 无意义的验证
assert len(labels) == len(prefixes)
assert len(prefixes) == len(suffixes)

In [10]:
def remove_html(s):
    """按照上面的规则，把html标签去除并且替换成markdown标签"""
    s = replace_cards(s)
    for label, prefix, suffix in zip(labels, prefixes, suffixes):
        s = re.sub(f"<{label}.*?>", prefix, s)
        s = re.sub(f"</{label}.*?>", suffix, s)
    s = re.sub('<br>', '\n', s)
    return s

In [11]:
def get_file(i, top_folder, now_folder):
    """从api获取内容、然后写入特定文件夹的.md文件"""
    # 这是固定的
    book_id = 20700413
    slug = df.loc[i, 'url']
    title = df.loc[i, 'title']
    url = f"https://www.yuque.com/api/docs/{slug}?book_id={book_id}"
    s = json.loads(requests.get(url).text)['data']['content']
    s = s.replace("<!doctype lake>", '')
    s = remove_html(s)
    with open(f"./{top_folder}/{now_folder}/{title}({slug}).md", 'w+') as f:
        f.write(s)
    print(f"./{top_folder}/{now_folder}/{title}({slug}).md", ' DONE')

In [13]:
for i in df.index:
    time.sleep(1)
    #### 这是一个错误写法 ####
    fail_count = 0     ####
    #### 这是一个错误写法 ####
    if fail_count > 20:
        break
    # 一级文件夹
    if df.loc[i, 'level'] == 0:
        top_floder = df.loc[i, 'title'].replace(' ', '_')
        os.system(f"mkdir {top_floder}")
    # 二级文件夹
    elif df.loc[i, 'level'] == 1:
        now_folder = df.loc[i, 'title'].replace(' ', '_')
        os.system(f"mkdir ./{top_floder}/{now_folder}")
    # 真题页面
    else:
        try:
            get_file(i, top_floder, now_folder)
        except Exception as e:
            fail_count += 1
            print(df.loc[i, 'title'], f' {i} ERROR ', repr(e))

茆pro派《五年考研三年模拟》习题集请关注“卡方训练营”公众号咨询小助手  2 ERROR  KeyError('code')
./茆pro派/北京大学_431/2022年北京大学431金融数学真题(tmi64a).md  DONE
./茆pro派/北京大学_431/2021年北京大学431金融数学真题(ng98pr).md  DONE
./茆pro派/北京大学_431/2020年北京大学431金融数学真题(oczqmf).md  DONE
./茆pro派/北京大学_431/2019年北京大学431金融数学真题(hiyrck).md  DONE
./茆pro派/北京大学_431/2018年北京大学431金融数学真题(pnuf3w).md  DONE
./茆pro派/北京大学_431/2017年北京大学431金融数学真题(wuqtgz).md  DONE
./茆pro派/北京大学_431/2016年北京大学431金融数学真题(kugqaf).md  DONE
./茆pro派/北京大学_432/2022年北京大学432应用统计真题(xgo24z).md  DONE
./茆pro派/北京大学_432/2021年北京大学432应用统计真题(qn0zit).md  DONE
./茆pro派/北京大学_432/2020年北京大学432应用统计真题(utwcx6).md  DONE
./茆pro派/北京大学_432/2019年北京大学432应用统计真题(cvs2vv).md  DONE
./茆pro派/北京大学_432/2018年北京大学432应用统计真题(fahq4l).md  DONE
./茆pro派/北京大学_432/2017年北京大学432应用统计真题(ing1x1).md  DONE
./茆pro派/北京大学_432/2016年北京大学432应用统计真题(fsak4n).md  DONE
./茆pro派/北京大学_432/2015年北京大学432应用统计真题(kgwt8c).md  DONE
./茆pro派/北京大学_432/2014年北京大学432应用统计真题(gl61tk).md  DONE
./茆pro派/中国人民大学_805/2022年中国人民大学805真题(qwvzim).md  DONE
./茆pro派/中国人民大学_805/2021年中国人民大学805真题

./纯茆派/东北师范大学432/2021年东北师范大学432应用统计真题(yu1p11).md  DONE
./纯茆派/东北师范大学432/2020年东北师范大学432应用统计真题(fz08os).md  DONE
./纯茆派/东北师范大学432/2019年东北师范大学432应用统计真题(hleouk).md  DONE
./纯茆派/东北师范大学432/2018年东北师范大学432应用统计真题(bntuua).md  DONE
./纯茆派/东北师范大学432/2017年东北师范大学432应用统计真题(qfnd7v).md  DONE
./纯茆派/东北师范大学432/2016年东北师范大学432应用统计真题(ydw4rt).md  DONE
./纯茆派/东北师范大学432/2015年东北师范大学432应用统计真题(ztszom).md  DONE
./纯茆派/东北师范大学432/2014年东北师范大学432应用统计真题(gseyg5).md  DONE
./纯茆派/东北师范大学432/2013年东北师范大学432应用统计真题(wuok1x).md  DONE
./纯茆派/东北师范大学432/2012年东北师范大学432应用统计真题(xeg5qh).md  DONE
2011年东北师范大学432应用统计真题  187 ERROR  KeyError('code')
./纯茆派/中国药科大学432/2022年中国药科大学432应用统计真题(my096q).md  DONE
./纯茆派/中国药科大学432/2021年中国药科大学432应用统计真题(mo77cx).md  DONE
./纯茆派/上海对外经贸大学805/2022年上海对外经贸大学805(bhgkro).md  DONE
./纯茆派/上海对外经贸大学805/2021年上海对外经贸大学805(px0iuq).md  DONE
./纯茆派/上海对外经贸大学805/2020年上海对外经贸大学805(sb2y5p).md  DONE
./纯茆派/上海对外经贸大学805/2019年上海对外经贸大学805(enxcak).md  DONE
./纯茆派/厦门大学868/2022年厦门大学868真题(ni80hu).md  DONE
./纯茆派/厦门大学868/2021年厦门大学868真题(fooqtg).md  DONE
./

./贾茆派/东华大学432/2022年东华大学432应用统计真题(btgnd4).md  DONE
./贾茆派/东华大学432/2021年东华大学432应用统计真题(xke84s).md  DONE
./贾茆派/东华大学432/2020年东华大学432应用统计真题(xoxnzh).md  DONE
./贾茆派/东华大学432/2019年东华大学432应用统计真题(xaubgb).md  DONE
./贾茆派/华东师范大学432/2022年华东师范大学432应用统计真题(hxeiu4).md  DONE
./贾茆派/华东师范大学432/2021年华东师范大学432应用统计真题(nsqut3).md  DONE
./贾茆派/华东师范大学432/2020年华东师范大学432应用统计真题(oem3se).md  DONE
./贾茆派/华东师范大学432/2019年华东师范大学432应用统计真题(wskb91).md  DONE
./贾茆派/华东师范大学432/2018年华东师范大学432应用统计真题(tty82c).md  DONE
./贾茆派/华东师范大学432/2017年华东师范大学432应用统计真题(mvbfa9).md  DONE
./贾茆派/华东师范大学432/2016年华东师范大学432应用统计真题(ac2dqq).md  DONE
./贾茆派/华东师范大学432/2015年华东师范大学432应用统计真题(cs8clr).md  DONE
./贾茆派/华东师范大学432/2014年华东师范大学432应用统计真题(kfd6nt).md  DONE
./贾茆派/华东师范大学432/2013年华东师范大学432应用统计真题(epgek3).md  DONE
./贾茆派/华东师范大学432/2012年华东师范大学432应用统计真题(vafygd).md  DONE
./贾茆派/华东师范大学432/2011年华东师范大学432应用统计真题(akdidk).md  DONE
./贾茆派/苏州大学432/2022年苏州大学432应用统计真题(dwi55w).md  DONE
./贾茆派/苏州大学432/2021年苏州大学432应用统计真题(aqsaur).md  DONE
./贾茆派/苏州大学432/2020年苏州大学432应用统计真题(dmsm6r).md  DONE
./

./贾茆派/中国石油大学（北京）432/2021年中国石油大学（北京）432应用统计真题(gyq6ea).md  DONE
./贾茆派/中国石油大学（北京）432/2020年中国石油大学（北京）432应用统计真题(de41lb).md  DONE
./贾茆派/中国石油大学（北京）432/2019年中国石油大学（北京）432应用统计真题(elxkvf).md  DONE
./贾茆派/中国地质大学（北京）432/2020年中国地质大学（北京）432应用统计真题(fuqoh2).md  DONE
贾茆派《五年考研三年模拟》习题集请关注“卡方训练营”公众号咨询小助手  543 ERROR  KeyError('code')
./纯贾派/中央财经大学432/2022年中央财经大学432应用统计真题(wdmzkh).md  DONE
./纯贾派/中央财经大学432/2021年中央财经大学432应用统计真题(rk5msn).md  DONE
2020年中央财经大学432应用统计真题  547 ERROR  KeyError('code')
./纯贾派/中央财经大学432/2019年中央财经大学432应用统计真题(wsg639).md  DONE
2018年中央财经大学432应用统计真题  549 ERROR  KeyError('code')
./纯贾派/中央财经大学432/2017年中央财经大学432应用统计真题(wy8uai).md  DONE
./纯贾派/中央财经大学432/2016年中央财经大学432应用统计真题(uue9at).md  DONE
./纯贾派/中央财经大学432/2015年中央财经大学432应用统计真题(zpiqrn).md  DONE
./纯贾派/中央财经大学432/2014年中央财经大学432应用统计真题(dz6o3e).md  DONE
./纯贾派/中央财经大学432/2013年中央财经大学432应用统计真题(uqyu9q).md  DONE
./纯贾派/中央财经大学432/2012年中央财经大学432应用统计真题(pkm3ez).md  DONE
./纯贾派/中央财经大学432/2011年中央财经大学432应用统计真题(lgqtgs).md  DONE
./纯贾派/内蒙古大学432/2021年内蒙古大学432应用统计真题(srz7r7).md  D

In [15]:
df.shape

(696, 4)

In [16]:
# 我傻了，本来是想让程序频繁出错的时候停下来的
#（因为）频繁的访问api很有可能被封！！
fail_count

0

# 转换成pdf

<font color=blue size=4>尝试了pandoc和用xelatex编译，但是爬下来的文件格式小错误太多了，编译效果都不尽如人意，就此作罢！用对语法比较宽容的markdown阅读器也可以达到很好的阅读效果！</font>

此外，仍然有一些不规范的html标签没有去除掉，不过留着也无伤大雅。

![](./assets/WechatIMG58.png)
![](./assets/WechatIMG59.png)
![](./assets/WechatIMG60.png)